In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import os.path as path
import time
import glob 
import numpy as np
import pandas as pd
import time
import json
from numpy import savez_compressed
from numpy import load

In [3]:
import netCDF4
import scipy
import json

In [4]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))
import FogDataNpzGen as FG
import FogDataloader as dataloader

In [5]:
FogDataConfigs = FG.FogData_Configs(input_path = None, 
                                     target_path = None, 
                                     start_date = FG.year_information['2009'][0], 
                                     finish_date = FG.year_information['2020'][1], 
                                     data_split_dict = FG.data_split_dict_[0], 
                                     data_straucture = '1D',
                                     lead_time_pred = 24, 
                                     vis_threshold = 1, 
                                     points_coords = FG.NAM_coords,  
                                     predictor_names = FG.NETCDF_PREDICTOR_NAMES['All']).return_config()

In [6]:
train_df, valid_df, test_df = FG.input_dataframe_generater(img_path = None, 
                                            target_path = None, 
                                            first_date_string = FogDataConfigs['start_date'], 
                                            last_date_string = FogDataConfigs['finish_date'], 
                                            target_binarizing_thre = FogDataConfigs['vis_threshold'], 
                                            year_split_dict = FogDataConfigs['data_split_dict']).split_data_train_valid_test()

In [7]:
train_df

,date_time,round_time,date_cycletime,date,cycletime,year,month,day,nam_nc_files_path,mur_nc_files_path,vis,vis_class,vis_category
0,20130101 05:55:00,20130101 06:00:00,20130101_0000,20130101,0000,2013,01,01,/data1/fog-data/fog-maps/2013/2013001/maps_201...,/data1/fog-data/fog-maps/2013/2013001/murs_201...,5.0,0,non_fog
1,20130101 11:55:00,20130101 12:00:00,20130101_0600,20130101,0600,2013,01,01,/data1/fog-data/fog-maps/2013/2013001/maps_201...,/data1/fog-data/fog-maps/2013/2013001/murs_201...,7.0,0,non_fog
2,20130101 17:55:00,20130101 18:00:00,20130101_1200,20130101,1200,2013,01,01,/data1/fog-data/fog-maps/2013/2013001/maps_201...,/data1/fog-data/fog-maps/2013/2013001/murs_201...,10.0,0,non_fog
3,20130101 23:55:00,20130102 00:00:00,20130101_1800,20130101,1800,2013,01,01,/data1/fog-data/fog-maps/2013/2013001/maps_201...,/data1/fog-data/fog-maps/2013/2013001/murs_201...,10.0,0,non_fog
4,20130104 05:55:00,20130104 06:00:00,20130104_0000,20130104,0000,2013,01,04,/data1/fog-data/fog-maps/2013/2013004/maps_201...,/data1/fog-data/fog-maps/2013/2013004/murs_201...,10.0,0,non_fog
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5455,20170823 23:55:00,20170824 00:00:00,20170823_1800,20170823,1800,2017,08,23,/data1/fog-data/fog-maps/2017/2017235/maps_201...,/data1/fog-data/fog-maps/2017/2017235/murs_201...,10.0,0,non_fog
5456,20170824 05:55:00,20170824 06:00:00,20170824_0000,20170824,0000,2017,08,24,/data1/fog-data/fog-maps/2017/2017236/maps_201...,/data1/fog-data/fog-maps/2017/2017236/murs_201...,10.0,0,non_fog
5457,20170824 11:55:00,20170824 12:00:00,20170824_0600,20170824,0600,2017,08,24,/data1/fog-data/fog-maps/2017/2017236/maps_201...,/data1/fog-data/fog-maps/2017/2017236/murs_201...,10.0,0,non_fog
5458,20170824 17:55:00,20170824 18:00:00,20170824_1200,20170824,1200,2017,08,24,/data1/fog-data/fog-maps/2017/2017236/maps_201...,/data1/fog-data/fog-maps/2017/2017236/murs_201...,10.0,0,non_fog


In [8]:
mean_std_dict_name = '/data1/fog/Hamid/SparkMET/EXPs/mean_std.json' 
with open(mean_std_dict_name, 'r') as file:
    norm_mean_std_dict = json.load(file)

#### train

In [10]:
full_timeseries_predictor_df = TabularFogDataGen(FogDataConfigs, train_df, mean_std_dict = norm_mean_std_dict).build()
full_timeseries_predictor_df

,TMP_2maboveground_P1_000,TMP_975mb_P1_000,TMP_950mb_P1_000,TMP_925mb_P1_000,TMP_900mb_P1_000,TMP_875mb_P1_000,TMP_850mb_P1_000,TMP_825mb_P1_000,TMP_800mb_P1_000,TMP_775mb_P1_000,...,VVEL_925mb_P2_024,VVEL_900mb_P2_024,VVEL_875mb_P2_024,VVEL_850mb_P2_024,VVEL_825mb_P2_024,VVEL_800mb_P2_024,VVEL_775mb_P2_024,VVEL_750mb_P2_024,VVEL_725mb_P2_024,VVEL_700mb_P2_024
0,-0.790110,-0.164659,0.094604,0.051437,-0.149572,-0.348887,0.193579,0.625990,0.257528,-0.108666,...,2.117955,1.760068,1.246708,0.911467,0.845206,-0.134372,-0.832278,-0.884217,-0.061746,0.970394
0,-0.816976,-0.189824,0.055231,0.264380,0.516854,0.265044,-0.085605,-0.335426,-0.482071,-0.741985,...,-1.694457,-0.819094,-0.440937,-0.568087,-0.494128,-0.735403,-0.888825,-1.061036,-1.551965,-1.368767
0,-0.792674,-0.359674,-0.417688,-0.387833,-0.242739,-0.061819,0.210480,-0.016438,-0.257343,-0.440939,...,0.410235,0.690329,1.494228,2.041124,1.560209,1.337146,0.966257,0.644705,0.045998,-0.582323
0,-1.766332,-2.142390,-2.084609,-1.582976,-1.073247,-1.001357,-0.979512,-0.865753,-0.884721,-0.958598,...,0.219198,0.951361,2.178610,3.227513,3.343414,2.618339,2.249436,2.177813,2.082919,1.243219
0,-2.980519,-3.368212,-2.993094,-2.768363,-2.766270,-2.965609,-3.331962,-3.113257,-2.858344,-2.595290,...,0.461750,0.183824,0.120494,-0.096793,-0.491454,-0.623281,-0.732280,-0.608197,-0.967706,-1.469643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2.303610,1.547901,1.325252,1.103053,0.890522,0.707574,0.654889,0.641111,0.618648,0.462929,...,14.279010,13.059058,12.180393,11.718767,11.831529,12.343453,12.921724,13.598202,14.352379,15.209593
0,1.574357,1.294227,1.249528,1.388359,1.333781,1.263947,1.165115,1.054022,0.941250,0.807382,...,2.757872,2.313507,1.838336,1.212344,0.824265,0.551447,0.325727,0.158246,-0.045597,-0.234244
0,0.846914,1.108536,0.953496,1.017565,1.082745,1.079525,1.025448,0.938053,0.804982,0.714693,...,-0.351495,-0.548247,-0.572942,-0.606111,-0.621486,-0.697893,-0.873879,-1.167265,-1.464269,-1.687899
0,0.487193,0.991764,0.836513,0.671470,0.645219,0.663128,0.610650,0.592713,0.549328,0.512282,...,13.972602,14.036557,14.787166,15.566390,16.484858,17.474068,18.372342,19.094084,19.495818,19.353295


In [9]:
this_df = pd.read_csv('/data1/fog/Dataset/train_tabular.csv', index_col= 0)
this_df

,date_time,round_time,date_cycletime,date,vis,vis_class,vis_category,TMP_2maboveground_P1_000,TMP_975mb_P1_000,TMP_950mb_P1_000,...,VVEL_925mb_P2_024,VVEL_900mb_P2_024,VVEL_875mb_P2_024,VVEL_850mb_P2_024,VVEL_825mb_P2_024,VVEL_800mb_P2_024,VVEL_775mb_P2_024,VVEL_750mb_P2_024,VVEL_725mb_P2_024,VVEL_700mb_P2_024
0,20130101 05:55:00,20130101 06:00:00,20130101_0000,20130101,5.0,0,non_fog,-0.790110,-0.164659,0.094604,...,2.117955,1.760068,1.246708,0.911467,0.845206,-0.134372,-0.832278,-0.884217,-0.061746,0.970394
0,20130101 05:55:00,20130101 06:00:00,20130101_0000,20130101,5.0,0,non_fog,-0.816976,-0.189824,0.055231,...,-1.694457,-0.819094,-0.440937,-0.568087,-0.494128,-0.735403,-0.888825,-1.061036,-1.551965,-1.368767
0,20130101 05:55:00,20130101 06:00:00,20130101_0000,20130101,5.0,0,non_fog,-0.792674,-0.359674,-0.417688,...,0.410235,0.690329,1.494228,2.041124,1.560209,1.337146,0.966257,0.644705,0.045998,-0.582323
0,20130101 05:55:00,20130101 06:00:00,20130101_0000,20130101,5.0,0,non_fog,-1.766332,-2.142390,-2.084609,...,0.219198,0.951361,2.178610,3.227513,3.343414,2.618339,2.249436,2.177813,2.082919,1.243219
0,20130101 05:55:00,20130101 06:00:00,20130101_0000,20130101,5.0,0,non_fog,-2.980519,-3.368212,-2.993094,...,0.461750,0.183824,0.120494,-0.096793,-0.491454,-0.623281,-0.732280,-0.608197,-0.967706,-1.469643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,20130101 05:55:00,20130101 06:00:00,20130101_0000,20130101,5.0,0,non_fog,2.303610,1.547901,1.325252,...,14.279010,13.059058,12.180393,11.718767,11.831529,12.343453,12.921724,13.598202,14.352379,15.209593
0,20130101 05:55:00,20130101 06:00:00,20130101_0000,20130101,5.0,0,non_fog,1.574357,1.294227,1.249528,...,2.757872,2.313507,1.838336,1.212344,0.824265,0.551447,0.325727,0.158246,-0.045597,-0.234244
0,20130101 05:55:00,20130101 06:00:00,20130101_0000,20130101,5.0,0,non_fog,0.846914,1.108536,0.953496,...,-0.351495,-0.548247,-0.572942,-0.606111,-0.621486,-0.697893,-0.873879,-1.167265,-1.464269,-1.687899
0,20130101 05:55:00,20130101 06:00:00,20130101_0000,20130101,5.0,0,non_fog,0.487193,0.991764,0.836513,...,13.972602,14.036557,14.787166,15.566390,16.484858,17.474068,18.372342,19.094084,19.495818,19.353295


In [11]:
train_df['date_time'].values

array(['20130101 05:55:00', '20130101 11:55:00', '20130101 17:55:00', ...,
       '20170824 11:55:00', '20170824 17:55:00', '20170824 23:55:00'],
      dtype=object)

In [13]:
this_df['date_time'] = train_df['date_time'].values
this_df['round_time'] = train_df['round_time'].values
this_df['date_cycletime'] = train_df['date_cycletime'].values
this_df['date'] = train_df['date'].values
this_df['vis'] = train_df['vis'].values
this_df['vis_class'] = train_df['vis_class'].values
this_df['vis_category'] = train_df['vis_category'].values

this_df

,date_time,round_time,date_cycletime,date,vis,vis_class,vis_category,TMP_2maboveground_P1_000,TMP_975mb_P1_000,TMP_950mb_P1_000,...,VVEL_925mb_P2_024,VVEL_900mb_P2_024,VVEL_875mb_P2_024,VVEL_850mb_P2_024,VVEL_825mb_P2_024,VVEL_800mb_P2_024,VVEL_775mb_P2_024,VVEL_750mb_P2_024,VVEL_725mb_P2_024,VVEL_700mb_P2_024
0,20130101 05:55:00,20130101 06:00:00,20130101_0000,20130101,5.0,0,non_fog,-0.790110,-0.164659,0.094604,...,2.117955,1.760068,1.246708,0.911467,0.845206,-0.134372,-0.832278,-0.884217,-0.061746,0.970394
0,20130101 11:55:00,20130101 12:00:00,20130101_0600,20130101,7.0,0,non_fog,-0.816976,-0.189824,0.055231,...,-1.694457,-0.819094,-0.440937,-0.568087,-0.494128,-0.735403,-0.888825,-1.061036,-1.551965,-1.368767
0,20130101 17:55:00,20130101 18:00:00,20130101_1200,20130101,10.0,0,non_fog,-0.792674,-0.359674,-0.417688,...,0.410235,0.690329,1.494228,2.041124,1.560209,1.337146,0.966257,0.644705,0.045998,-0.582323
0,20130101 23:55:00,20130102 00:00:00,20130101_1800,20130101,10.0,0,non_fog,-1.766332,-2.142390,-2.084609,...,0.219198,0.951361,2.178610,3.227513,3.343414,2.618339,2.249436,2.177813,2.082919,1.243219
0,20130104 05:55:00,20130104 06:00:00,20130104_0000,20130104,10.0,0,non_fog,-2.980519,-3.368212,-2.993094,...,0.461750,0.183824,0.120494,-0.096793,-0.491454,-0.623281,-0.732280,-0.608197,-0.967706,-1.469643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,20170823 23:55:00,20170824 00:00:00,20170823_1800,20170823,10.0,0,non_fog,2.303610,1.547901,1.325252,...,14.279010,13.059058,12.180393,11.718767,11.831529,12.343453,12.921724,13.598202,14.352379,15.209593
0,20170824 05:55:00,20170824 06:00:00,20170824_0000,20170824,10.0,0,non_fog,1.574357,1.294227,1.249528,...,2.757872,2.313507,1.838336,1.212344,0.824265,0.551447,0.325727,0.158246,-0.045597,-0.234244
0,20170824 11:55:00,20170824 12:00:00,20170824_0600,20170824,10.0,0,non_fog,0.846914,1.108536,0.953496,...,-0.351495,-0.548247,-0.572942,-0.606111,-0.621486,-0.697893,-0.873879,-1.167265,-1.464269,-1.687899
0,20170824 17:55:00,20170824 18:00:00,20170824_1200,20170824,10.0,0,non_fog,0.487193,0.991764,0.836513,...,13.972602,14.036557,14.787166,15.566390,16.484858,17.474068,18.372342,19.094084,19.495818,19.353295


In [19]:
this_df.to_csv('/data1/fog/Dataset/train_tabular.csv')

#### valid

In [24]:
full_timeseries_predictor_valid_df = TabularFogDataGen(FogDataConfigs, valid_df, mean_std_dict = norm_mean_std_dict).build()
full_timeseries_predictor_valid_df

,TMP_2maboveground_P1_000,TMP_975mb_P1_000,TMP_950mb_P1_000,TMP_925mb_P1_000,TMP_900mb_P1_000,TMP_875mb_P1_000,TMP_850mb_P1_000,TMP_825mb_P1_000,TMP_800mb_P1_000,TMP_775mb_P1_000,...,VVEL_925mb_P2_024,VVEL_900mb_P2_024,VVEL_875mb_P2_024,VVEL_850mb_P2_024,VVEL_825mb_P2_024,VVEL_800mb_P2_024,VVEL_775mb_P2_024,VVEL_750mb_P2_024,VVEL_725mb_P2_024,VVEL_700mb_P2_024
0,-1.086830,-0.736516,-0.534827,-0.307332,-0.396852,-0.327698,-0.273327,-0.205877,-0.087309,-0.098171,...,-0.933610,-1.388926,-1.745157,-1.526601,-1.403847,-1.735017,-1.554315,-0.866982,-0.736410,-0.645890
0,-1.147146,-0.135918,-0.042248,-0.226547,-0.521599,-0.159794,0.061880,0.138754,0.130164,-0.086173,...,-3.515833,-4.422708,-5.301536,-6.619379,-7.488660,-7.892413,-8.039596,-7.909092,-7.799493,-7.357260
0,-1.429266,-0.436430,-0.362771,-0.162433,-0.091641,0.043168,0.175918,0.258258,0.083242,-0.293836,...,4.675964,4.878689,4.690756,3.688812,2.501532,1.528800,0.153374,-0.768479,-1.596708,-2.291265
0,-0.106161,-0.410077,-0.278041,-0.060787,0.062673,0.191338,0.229741,0.153913,-0.104667,-0.439012,...,-3.942327,-3.229562,-2.568365,-1.598125,-0.590777,0.254465,0.850403,1.243010,1.654085,1.148516
0,-0.859302,-0.161173,0.156461,0.038992,0.000825,0.239068,0.208374,0.225659,0.154828,-0.168697,...,1.790687,2.459811,2.589036,2.350682,1.746420,1.201680,0.265516,-0.140014,-0.341144,-0.785856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,-1.791557,-2.410447,-2.605662,-2.768876,-2.918049,-2.798353,-2.175403,-2.341681,-1.973662,-1.465181,...,-2.180051,-1.676643,-1.502214,-0.820271,-0.374218,-0.054603,-0.454092,-0.228537,-0.064894,-0.665235
0,-1.470074,-1.722406,-1.801356,-1.916720,-2.194159,-2.386674,-2.352311,-2.396203,-2.118836,-1.338644,...,-0.423243,0.411073,0.205031,-1.018134,-2.212834,-2.369721,-1.902155,-0.823746,1.414646,3.971854
0,-1.278306,-1.153650,-1.301880,-1.384220,-1.515079,-1.651580,-1.877921,-2.125613,-1.705951,-0.922589,...,-0.442309,0.083548,0.994171,1.821363,2.245774,2.020697,1.657181,1.132568,0.961575,0.219106
0,-1.020973,-0.718366,-0.829688,-0.902531,-1.006587,-1.102486,-1.255055,-1.503166,-1.526569,-0.838920,...,-5.696106,-6.922573,-7.895815,-7.794600,-6.716245,-6.056485,-5.400547,-4.435709,-3.761468,-2.748775


In [27]:
full_timeseries_predictor_valid_df['date_time'] = valid_df['date_time'].values
full_timeseries_predictor_valid_df['round_time'] = valid_df['round_time'].values
full_timeseries_predictor_valid_df['date_cycletime'] = valid_df['date_cycletime'].values
full_timeseries_predictor_valid_df['date'] = valid_df['date'].values
full_timeseries_predictor_valid_df['vis'] = valid_df['vis'].values
full_timeseries_predictor_valid_df['vis_class'] = valid_df['vis_class'].values
full_timeseries_predictor_valid_df['vis_category'] = valid_df['vis_category'].values

full_timeseries_predictor_valid_df

,date_time,round_time,date_cycletime,date,vis,vis_class,vis_category,TMP_2maboveground_P1_000,TMP_975mb_P1_000,TMP_950mb_P1_000,...,VVEL_925mb_P2_024,VVEL_900mb_P2_024,VVEL_875mb_P2_024,VVEL_850mb_P2_024,VVEL_825mb_P2_024,VVEL_800mb_P2_024,VVEL_775mb_P2_024,VVEL_750mb_P2_024,VVEL_725mb_P2_024,VVEL_700mb_P2_024
0,20090102 06:05:00,20090102 06:00:00,20090102_0000,20090102,7.00,0,non_fog,-1.086830,-0.736516,-0.534827,...,-0.933610,-1.388926,-1.745157,-1.526601,-1.403847,-1.735017,-1.554315,-0.866982,-0.736410,-0.645890
0,20090102 12:05:00,20090102 12:00:00,20090102_0600,20090102,0.25,1,fog,-1.147146,-0.135918,-0.042248,...,-3.515833,-4.422708,-5.301536,-6.619379,-7.488660,-7.892413,-8.039596,-7.909092,-7.799493,-7.357260
0,20090102 18:05:00,20090102 18:00:00,20090102_1200,20090102,0.25,1,fog,-1.429266,-0.436430,-0.362771,...,4.675964,4.878689,4.690756,3.688812,2.501532,1.528800,0.153374,-0.768479,-1.596708,-2.291265
0,20090103 00:05:00,20090103 00:00:00,20090102_1800,20090102,4.00,0,non_fog,-0.106161,-0.410077,-0.278041,...,-3.942327,-3.229562,-2.568365,-1.598125,-0.590777,0.254465,0.850403,1.243010,1.654085,1.148516
0,20090103 06:05:00,20090103 06:00:00,20090103_0000,20090103,4.00,0,non_fog,-0.859302,-0.161173,0.156461,...,1.790687,2.459811,2.589036,2.350682,1.746420,1.201680,0.265516,-0.140014,-0.341144,-0.785856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,20121230 23:55:00,20121231 00:00:00,20121230_1800,20121230,10.00,0,non_fog,-1.791557,-2.410447,-2.605662,...,-2.180051,-1.676643,-1.502214,-0.820271,-0.374218,-0.054603,-0.454092,-0.228537,-0.064894,-0.665235
0,20121231 05:55:00,20121231 06:00:00,20121231_0000,20121231,10.00,0,non_fog,-1.470074,-1.722406,-1.801356,...,-0.423243,0.411073,0.205031,-1.018134,-2.212834,-2.369721,-1.902155,-0.823746,1.414646,3.971854
0,20121231 11:55:00,20121231 12:00:00,20121231_0600,20121231,7.00,0,non_fog,-1.278306,-1.153650,-1.301880,...,-0.442309,0.083548,0.994171,1.821363,2.245774,2.020697,1.657181,1.132568,0.961575,0.219106
0,20121231 17:55:00,20121231 18:00:00,20121231_1200,20121231,2.50,0,non_fog,-1.020973,-0.718366,-0.829688,...,-5.696106,-6.922573,-7.895815,-7.794600,-6.716245,-6.056485,-5.400547,-4.435709,-3.761468,-2.748775


In [28]:
full_timeseries_predictor_valid_df.to_csv('/data1/fog/Dataset/valid_tabular.csv')

#### test

In [17]:
full_timeseries_predictor_test_df = TabularFogDataGen(FogDataConfigs, test_df, mean_std_dict = norm_mean_std_dict).build()
full_timeseries_predictor_test_df

,TMP_2maboveground_P1_000,TMP_975mb_P1_000,TMP_950mb_P1_000,TMP_925mb_P1_000,TMP_900mb_P1_000,TMP_875mb_P1_000,TMP_850mb_P1_000,TMP_825mb_P1_000,TMP_800mb_P1_000,TMP_775mb_P1_000,...,VVEL_925mb_P2_024,VVEL_900mb_P2_024,VVEL_875mb_P2_024,VVEL_850mb_P2_024,VVEL_825mb_P2_024,VVEL_800mb_P2_024,VVEL_775mb_P2_024,VVEL_750mb_P2_024,VVEL_725mb_P2_024,VVEL_700mb_P2_024
0,1.153767,1.037438,0.836368,0.981121,1.211519,1.280738,1.456452,1.707018,2.127524,2.588875,...,0.046351,0.070160,-0.188060,-0.133046,-0.091460,-0.122058,-0.095060,-0.077938,0.065223,0.485295
0,0.646428,0.821626,0.624201,1.102719,1.164284,1.188602,1.623523,1.898614,2.311543,2.731099,...,-0.995472,-0.803791,-0.472449,-0.174277,0.153436,0.449170,0.700687,0.889936,1.112947,1.519979
0,0.666592,0.793947,0.626436,0.429794,1.078277,1.435273,1.486121,1.834826,2.348879,2.595846,...,-0.378401,0.019148,0.202263,0.069537,-0.099108,-0.199526,-0.291873,-0.321096,-0.236666,0.085646
0,1.629502,1.050555,0.830985,0.609846,0.636263,1.025628,1.390056,1.968411,2.556708,2.744182,...,0.972434,1.626225,1.013108,0.309727,-0.107101,-0.388934,-0.442141,-0.372575,-0.370218,-0.391289
0,1.113074,0.965435,0.783753,0.657740,1.249123,1.451319,1.881333,2.482646,2.916474,2.847010,...,0.754454,0.829264,0.817437,0.795890,0.953379,1.157614,1.457150,1.789261,2.078580,2.159481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1.483457,0.911786,0.699023,0.643763,0.741338,0.717024,0.657133,0.716169,0.641711,0.358694,...,0.663139,0.241094,-0.226367,-0.566937,-1.076689,-1.450107,-0.903743,0.258026,1.458047,2.133400
0,1.202005,1.032830,1.288198,1.197154,1.173922,1.182006,1.143401,1.183277,1.295402,1.317150,...,-4.572956,-4.145052,-3.083345,-1.810185,-0.380381,1.004645,1.892198,2.064401,1.758035,1.302179
0,0.150874,0.950698,1.036712,1.510958,1.377638,1.171525,0.958024,0.921199,0.730007,0.465985,...,-4.379259,-3.576052,-2.809354,-2.060515,-1.318911,-0.751364,-0.361361,-0.006923,0.270794,0.370084
0,0.013639,0.422601,0.577768,0.693498,0.654708,0.533934,0.456216,0.374896,0.261919,0.167161,...,-0.213091,-0.162815,-0.149376,-0.023383,0.288669,0.745015,0.939422,0.580541,-0.085163,-0.603256


In [29]:
test_df_old = pd.read_csv('/data1/fog/Dataset/test_tabular.csv', index_col= 0)

In [30]:
test_df_old['date_time'] = test_df['date_time'].values
test_df_old['round_time'] = test_df['round_time'].values
test_df_old['date_cycletime'] = test_df['date_cycletime'].values
test_df_old['date'] = test_df['date'].values
test_df_old['vis'] = test_df['vis'].values
test_df_old['vis_class'] = test_df['vis_class'].values
test_df_old['vis_category'] = test_df['vis_category'].values

test_df_old

,date_time,round_time,date_cycletime,date,vis,vis_class,vis_category,TMP_2maboveground_P1_000,TMP_975mb_P1_000,TMP_950mb_P1_000,...,VVEL_925mb_P2_024,VVEL_900mb_P2_024,VVEL_875mb_P2_024,VVEL_850mb_P2_024,VVEL_825mb_P2_024,VVEL_800mb_P2_024,VVEL_775mb_P2_024,VVEL_750mb_P2_024,VVEL_725mb_P2_024,VVEL_700mb_P2_024
0,20180530 05:55:00,20180530 06:00:00,20180530_0000,20180530,10.0,0,non_fog,1.153767,1.037438,0.836368,...,0.046351,0.070160,-0.188060,-0.133046,-0.091460,-0.122058,-0.095060,-0.077938,0.065223,0.485295
0,20180530 11:55:00,20180530 12:00:00,20180530_0600,20180530,10.0,0,non_fog,0.646428,0.821626,0.624201,...,-0.995472,-0.803791,-0.472449,-0.174277,0.153436,0.449170,0.700687,0.889936,1.112947,1.519979
0,20180530 17:55:00,20180530 18:00:00,20180530_1200,20180530,10.0,0,non_fog,0.666592,0.793947,0.626436,...,-0.378401,0.019148,0.202263,0.069537,-0.099108,-0.199526,-0.291873,-0.321096,-0.236666,0.085646
0,20180530 23:55:00,20180531 00:00:00,20180530_1800,20180530,10.0,0,non_fog,1.629502,1.050555,0.830985,...,0.972434,1.626225,1.013108,0.309727,-0.107101,-0.388934,-0.442141,-0.372575,-0.370218,-0.391289
0,20180531 05:55:00,20180531 06:00:00,20180531_0000,20180531,10.0,0,non_fog,1.113074,0.965435,0.783753,...,0.754454,0.829264,0.817437,0.795890,0.953379,1.157614,1.457150,1.789261,2.078580,2.159481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,20200527 23:55:00,20200528 00:00:00,20200527_1800,20200527,10.0,0,non_fog,1.483457,0.911786,0.699023,...,0.663139,0.241094,-0.226367,-0.566937,-1.076689,-1.450107,-0.903743,0.258026,1.458047,2.133400
0,20200529 05:55:00,20200529 06:00:00,20200529_0000,20200529,10.0,0,non_fog,1.202005,1.032830,1.288198,...,-4.572956,-4.145052,-3.083345,-1.810185,-0.380381,1.004645,1.892198,2.064401,1.758035,1.302179
0,20200529 11:55:00,20200529 12:00:00,20200529_0600,20200529,10.0,0,non_fog,0.150874,0.950698,1.036712,...,-4.379259,-3.576052,-2.809354,-2.060515,-1.318911,-0.751364,-0.361361,-0.006923,0.270794,0.370084
0,20200529 17:55:00,20200529 18:00:00,20200529_1200,20200529,10.0,0,non_fog,0.013639,0.422601,0.577768,...,-0.213091,-0.162815,-0.149376,-0.023383,0.288669,0.745015,0.939422,0.580541,-0.085163,-0.603256


In [31]:
test_df_old.to_csv('/data1/fog/Dataset/test_tabular.csv')

In [23]:
class TabularFogDataGen():
    def __init__(self, FogDataConfigs,
                 dataframe,
                 mean_std_dict: dict,
                 ):
        

        self.dataframe = dataframe
        self.mean_std_dict = mean_std_dict
        self.lead_time_pred  = FogDataConfigs['lead_time_pred']
        self.predictor_names = FogDataConfigs['predictor_names']
        self.data_straucture = FogDataConfigs['data_straucture']
        self.point_geolocation_dic = FogDataConfigs['points_coords']

    def build(self):
        
        full_timeseries_predictor_df = None
        for idx, row in self.dataframe.iterrows(): 
            date_time      = row['date_time']
            round_time     = row['round_time']
            date_cycletime = row['date_cycletime']
            visibility     = row['vis']

            # reading the nam map
            nc_nam_timeseries_files_path_list = row['nam_nc_files_path']


            timeseries_nam_predictor_matrix = self.read_nc_nam_maps(nc_nam_timeseries_files_path_list)

            if full_timeseries_predictor_df is None:
                full_timeseries_predictor_df = timeseries_nam_predictor_matrix

            else:
                full_timeseries_predictor_df = pd.concat((
                    full_timeseries_predictor_df, timeseries_nam_predictor_matrix), axis=0)
            

        return full_timeseries_predictor_df


    def read_nc_nam_maps(self, netcdf_file_root_names): 
        
        if self.lead_time_pred == 6:

            netcdf_file_006_names  = netcdf_file_root_names[0:57] + '006_input.nc'
            netcdf_file_names_list = [netcdf_file_root_names, netcdf_file_006_names]
            lead_time_steps = ['000', '006']

        elif self.lead_time_pred == 12:

            netcdf_file_006_names = netcdf_file_root_names[0:57] + '006_input.nc'
            netcdf_file_009_names = netcdf_file_root_names[0:57] + '009_input.nc'
            netcdf_file_012_names = netcdf_file_root_names[0:57] + '012_input.nc'
            netcdf_file_names_list = [netcdf_file_root_names, netcdf_file_006_names, netcdf_file_009_names, netcdf_file_012_names]
            lead_time_steps = ['000', '006', '009', '012']

        elif self.lead_time_pred == 24:

            netcdf_file_006_names = netcdf_file_root_names[0:57] + '006_input.nc'
            netcdf_file_012_names = netcdf_file_root_names[0:57] + '012_input.nc'
            netcdf_file_024_names = netcdf_file_root_names[0:57] + '024_input.nc'
            netcdf_file_names_list = [netcdf_file_root_names, netcdf_file_006_names, netcdf_file_012_names, netcdf_file_024_names]
            lead_time_steps = ['000', '006', '012', '024']

        timeseries_predictor_matrix = None

        for idx, nc_file_name in enumerate(netcdf_file_names_list):
            predictor_names_updated = self.predictor_names
            dataset_object = netCDF4.Dataset(nc_file_name)

            nam_predictor_matrix = None
            
            for this_predictor_name in predictor_names_updated:

                this_predictor_matrix = np.array(
                    dataset_object.variables[this_predictor_name][:], dtype=float
                )
                this_predictor_matrix = np.expand_dims(
                    this_predictor_matrix, axis=-1)
                
                this_predictor_mean  = self.mean_std_dict[this_predictor_name][0]
                this_predictor_std   = self.mean_std_dict[this_predictor_name][1] 

                this_predictor_matrix = (this_predictor_matrix - this_predictor_mean)/this_predictor_std

                if nam_predictor_matrix is None:
                    nam_predictor_matrix = this_predictor_matrix + 0.
                else:
                    nam_predictor_matrix = np.concatenate(
                        (nam_predictor_matrix, this_predictor_matrix), axis=-1
                    )

            time_steps = lead_time_steps[idx]
            this_timeseries_predictor_matrix = self.return_tabular_data(nam_predictor_matrix, time_steps) 

            if timeseries_predictor_matrix is None:
                timeseries_predictor_matrix = this_timeseries_predictor_matrix

            else:
                timeseries_predictor_matrix = pd.concat((
                    timeseries_predictor_matrix, this_timeseries_predictor_matrix), axis=1)
    

        return timeseries_predictor_matrix

    def return_tabular_data(self, nam_predictor_matrix, time_steps):
            
        output_dfs = []

        for key in self.point_geolocation_dic:
            i = self.point_geolocation_dic[key][0]
            j = self.point_geolocation_dic[key][1]
            this_point_df = pd.DataFrame() 
            for idx in range(nam_predictor_matrix.shape[3]):

                this_predictor_matrix = nam_predictor_matrix[0, :, :, idx] 
                feature_value         = this_predictor_matrix[i, j]
                feature_name          = self.predictor_names[idx] + '_' + key + '_' + time_steps

                this_point_df.at[0, feature_name] = feature_value
            
            output_dfs.append(this_point_df)
        
        output = pd.concat(output_dfs, axis=1)

        return output 


